### Hydraulic Design Calculations

The following calculations determine the required vessel dimensions based on hydraulic constraints:

In [ ]:
# Service Flow Rate Design (SAC)
# Using standard heuristics for RO pretreatment

from math import pi

# Design parameters (from simulation inputs)
Q_m3_hr = design.get('flow_m3_hr', 100)      # Design flow rate in m³/hr
SV_hr = 16                                    # Service velocity (bed volumes/hour)
LV_max_m_hr = 25                             # Maximum linear velocity in m/hr

# Calculate required bed volume
V_bed_m3 = Q_m3_hr / SV_hr                   # Bed volume required in m³

# Calculate minimum cross-sectional area
A_min_m2 = Q_m3_hr / LV_max_m_hr             # Minimum area for linear velocity in m²

# Convert to liters for display
V_bed_L = V_bed_m3 * 1000                    # Volume in liters

print(f"Design flow rate Q = {Q_m3_hr:.1f} m³/hr")
print(f"Required bed volume = {V_bed_m3:.2f} m³ = {V_bed_L:.0f} L")
print(f"Minimum cross-sectional area = {A_min_m2:.2f} m²")

In [ ]:
# Vessel Diameter Selection
# Standard vessel diameters for containerized systems

# Minimum diameter from area
D_min_m = (4 * A_min_m2 / pi) ** 0.5

# Round up to standard diameter (0.1m increments)
D_selected_m = 2.3                           # Selected standard diameter in meters

# Actual cross-sectional area
A_actual_m2 = pi * (D_selected_m / 2) ** 2

# Actual linear velocity
LV_actual_m_hr = Q_m3_hr / A_actual_m2

print(f"Minimum diameter = {D_min_m:.2f} m")
print(f"Selected standard diameter = {D_selected_m:.2f} m")
print(f"Actual linear velocity = {LV_actual_m_hr:.1f} m/hr")

In [ ]:
# Bed Depth Calculation

# Calculate bed depth from volume
h_bed_m = V_bed_m3 / A_actual_m2

# Minimum bed depth constraint
h_bed_min_m = 0.75                           # Minimum for proper distribution

# Select actual bed depth (round to 0.05m)
h_bed_final_m = 1.5                          # Selected bed depth in meters

# Recalculate actual bed volume
V_bed_actual_m3 = A_actual_m2 * h_bed_final_m

# Service flow rate in BV/hr
SV_actual_hr = Q_m3_hr / V_bed_actual_m3

print(f"Calculated bed depth = {h_bed_m:.2f} m")
print(f"Selected bed depth = {h_bed_final_m:.2f} m")
print(f"Actual bed volume = {V_bed_actual_m3:.2f} m³")
print(f"Actual service velocity = {SV_actual_hr:.1f} BV/hr")

In [ ]:
# Freeboard and Total Height

# Freeboard for backwash expansion (100% for conservative design)
expansion = 1.0                               # 100% expansion factor
h_freeboard_m = h_bed_final_m * expansion

# Total vessel height
h_vessel_m = h_bed_final_m + h_freeboard_m + 0.1  # Plus distributor/collector space

# Aspect ratio check
aspect_ratio = h_bed_final_m / D_selected_m

print(f"Freeboard height = {h_freeboard_m:.2f} m")
print(f"Total vessel height = {h_vessel_m:.2f} m")
print(f"Aspect ratio (L/D) = {aspect_ratio:.2f}")

In [ ]:
# Summary of vessel design
vessel_summary = [
    ['Vessel Diameter', f"{D_selected_m:.2f}", 'm'],
    ['Bed Depth', f"{h_bed_final_m:.2f}", 'm'],
    ['Bed Volume', f"{V_bed_actual_m3:.2f}", 'm³'],
    ['Freeboard Height', f"{h_freeboard_m:.2f}", 'm'],
    ['Total Vessel Height', f"{h_vessel_m:.2f}", 'm'],
    ['Linear Velocity', f"{LV_actual_m_hr:.1f}", 'm/hr'],
    ['Service Velocity', f"{SV_actual_hr:.1f}", 'BV/hr'],
    ['Aspect Ratio (L/D)', f"{aspect_ratio:.2f}", '-']
]

df_vessel = pd.DataFrame(vessel_summary, columns=['Parameter', 'Value', 'Units'])
display(df_vessel.style.set_caption('SAC Vessel Design Summary'))

# Design verification
print("\n✓ Design Verification:")
if LV_actual_m_hr <= 25:
    print(f"  Linear velocity {LV_actual_m_hr:.1f} ≤ 25 m/hr ✓")
else:
    print(f"  ⚠️ Linear velocity {LV_actual_m_hr:.1f} > 25 m/hr")

if h_bed_final_m >= 0.75:
    print(f"  Bed depth {h_bed_final_m:.2f} ≥ 0.75 m ✓")
else:
    print(f"  ⚠️ Bed depth {h_bed_final_m:.2f} < 0.75 m")

if D_selected_m <= 2.4:
    print(f"  Diameter {D_selected_m:.2f} ≤ 2.4 m (fits in container) ✓")
else:
    print(f"  ⚠️ Diameter {D_selected_m:.2f} > 2.4 m (exceeds container)")